In [1]:
import numpy as np
import pandas as pd
import torch
# from logs import log
from tqdm.notebook import tqdm
# from tqdm import tqdm
import networkx as nx
import os
import torch.nn as nn
import torch_geometric.transforms as T
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GAE, VGAE, APPNP
from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import from_networkx, negative_sampling, to_networkx
from sklearn.metrics import roc_auc_score, average_precision_score
from model import LinkPred

### WalkPooling model version logs:
    * ver1: WalkPooling first try. 

In [2]:
"""Datasets:
    * id: edge id, 
    * from & to: 'from' node point to 'to' node, 
    * label: connect or not.
    * content: containing each node's attribute.

   Evaluate:
    * AUC: area under ROC curve
    * AP: average precision
"""
data_path = '../dataset1/'
store_file = 'WalkPool_ver1_submission'
# log_file = 'logs/'+store_file+'.log'
# logger = log(path=data_path, file=log_file)

df_train = pd.read_csv(data_path+'raw/train.csv').sort_values('from')
df_test = pd.read_csv(data_path+'raw/test.csv')
df_content = pd.read_csv(data_path+'raw/content.csv', delimiter='\t', header=None)
df_upload = pd.read_csv(data_path+'raw/upload.csv')

In [3]:
class Graph_dataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(Graph_dataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
    
    @property
    def raw_file_names(self):
        return ['train.csv', 'content.csv']
    
    @property
    def processed_file_names(self):
        return ['train.pt']
    
    def download(self):
        pass

    def process(self):
        self.data = pd.read_csv(self.raw_paths[0]).sort_values('from')
        node_feats = pd.read_csv(self.raw_paths[1], delimiter='\t', header=None, index_col=0)
        
        # Get node features. [num_nodes, num_node_features]
        x = torch.tensor(node_feats.sort_index().values, dtype=torch.float)
        
        # Get positive data.(label = 1: link)
        pos_data = self.data[self.data['label'] == 1]
        # neg_data = self.data[self.data['label'] == 0]

        # Get edge index.
        graph = nx.from_pandas_edgelist(pos_data, 'from', 'to', edge_attr=None)

        pair1 = [i[0] for i in graph.edges()]
        pair2 = [i[1] for i in graph.edges()]
        pos_edge_index = torch.LongTensor([pair1+pair2,pair2+pair1])

        # Create Data object.
        proc_graph = Data(x=x,
                          edge_index=pos_edge_index,
                          y=None)
        print(proc_graph)

        data, slices = self.collate([proc_graph])
        torch.save((data, slices), self.processed_paths[0])

In [4]:
demo = Graph_dataset(data_path)
data = demo.data
data = T.NormalizeFeatures()(data)
train_data, val_data, test_data = T.RandomLinkSplit(num_val=0.05, 
                                                    num_test=0.1, 
                                                    split_labels=True, 
                                                    is_undirected=True, 
                                                    add_negative_train_samples=False
                                                    )(data)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\torch_geometric\data\in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [5]:
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        hidden_channels=64
        self.conv1 = GCNConv(in_channels, hidden_channels, cached=True)
        self.conv_mu = GCNConv(hidden_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(hidden_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)
    
def compute_scores(z,test_pos,test_neg):
    test = torch.cat((test_pos, test_neg),dim=1)
    labels = torch.zeros(test.size(1),1)
    labels[0:test_pos.size(1)] = 1
    row, col = test
    src = z[row]
    tgt = z[col]
    scores = torch.sigmoid(torch.sum(src * tgt,dim=1))
    auc = roc_auc_score(labels, scores)
    ap = average_precision_score(labels, scores)
    return auc,ap

def CalVGAE(edge_index, x, test_and_val, embedding_dim):
    print('___Calculating VGAE embbeding___')
    test_pos,test_neg,val_pos,val_neg=test_and_val
    out_channels = int(embedding_dim)
    num_features = x.size(1)
    model = VGAE(VariationalGCNEncoder(num_features, out_channels)).to(device)
    edge_index = edge_index.to(device)
    x = x.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    num_nodes = torch.max(edge_index)
    best_val_auc=0
    for epoch in range(1, 500 + 1):
        model.train()
        optimizer.zero_grad()
        z = model.encode(x, edge_index)
        loss = model.recon_loss(z, edge_index)
        loss = loss + (1 / num_nodes) * model.kl_loss()
        loss.backward()
        optimizer.step()
        if epoch%10 == 0:
            model.eval()
            z = model.encode(x, edge_index)
            z = z.cpu().clone().detach()
            auc,_=compute_scores(z,val_pos,val_neg)
            if auc>best_val_auc:
                best_val_auc=auc
                record_z=z.clone().detach()
            print(f'Setp: {epoch:03d} /500, Loss : {loss.item():.4f}, Val_auc:{best_val_auc:.4f}')
            
    auc,ap=compute_scores(record_z,test_pos,test_neg)
    
    print(f'vgae prediction accuracy, AUC: {auc:.4f}, AP: {ap:.4f}')
    del model
    torch.cuda.empty_cache()
    return record_z, auc, ap

In [6]:
def set_init_attribute_representation(train_data, val_data, test_data):
    data_observed = Data(edge_index=train_data.edge_index)
    data_observed.num_nodes = train_data.num_nodes
    
    edge_index_observed = data_observed.edge_index
    
    x = train_data.x
    val_and_test = [test_data.pos_edge_label_index, test_data.neg_edge_label_index, val_data.pos_edge_label_index, val_data.neg_edge_label_index]
    num_nodes, _ = x.shape

    if (num_nodes-1) in edge_index_observed:
        # .clone().detach() info. https://blog.csdn.net/Answer3664/article/details/104417013
        edge_index_observed=edge_index_observed.clone().detach()
    else:
        edge_index_observed=torch.cat((edge_index_observed.clone().detach(),torch.tensor([[num_nodes-1],[num_nodes-1]])),dim=1)

    data_observed.x, auc, ap = CalVGAE(edge_index_observed, x, val_and_test, embedding_dim=32)
    feature_results = [auc, ap]

    return data_observed, feature_results

In [7]:
import scipy.sparse as ssp
from scipy.sparse.csgraph import shortest_path
# Drnl feature in the SEAL paper
def drnl_node_labeling(edge_index, src, dst, num_nodes):

    edge_weight = torch.ones(edge_index.size(1), dtype=int)
    adj = ssp.csr_matrix(
            (edge_weight, (edge_index[0], edge_index[1])), 
            shape=(num_nodes, num_nodes))
    # Double Radius Node Labeling (DRNL).
    src, dst = (dst, src) if src > dst else (src, dst)

    idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
    adj_wo_src = adj[idx, :][:, idx]

    idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
    adj_wo_dst = adj[idx, :][:, idx]

    dist2src = shortest_path(adj_wo_dst, directed=False, unweighted=True, indices=src)
    dist2src = np.insert(dist2src, dst, 0, axis=0)
    dist2src = torch.from_numpy(dist2src)

    dist2dst = shortest_path(adj_wo_src, directed=False, unweighted=True, indices=dst-1)
    dist2dst = np.insert(dist2dst, src, 0, axis=0)
    dist2dst = torch.from_numpy(dist2dst)

    dist = dist2src + dist2dst
    dist_over_2, dist_mod_2 = dist // 2, dist % 2

    z = 1 + torch.min(dist2src, dist2dst)
    z += dist_over_2 * (dist_over_2 + dist_mod_2 - 1)
    z[src] = 1.
    z[dst] = 1.
    z[torch.isnan(z)] = 0.
    return z.to(torch.int)

In [8]:
train_data.pos_edge_label_index

tensor([[ 374, 1048, 1537,  ..., 1601,  625,  387],
        [2506, 2180, 2273,  ..., 2695, 1325, 2405]])

In [9]:
def k_hop_subgraph(node_idx, num_hops, edge_index, max_nodes_per_hop = None, num_nodes = None):
    row, col = edge_index
    node_mask = row.new_empty(num_nodes, dtype=torch.bool)
    edge_mask = row.new_empty(row.size(0), dtype=torch.bool)

    node_idx = node_idx.to(row.device)

    subsets = [node_idx]

    not_visited = row.new_empty(num_nodes, dtype=torch.bool)
    not_visited.fill_(True)

    for _ in range(num_hops):
        node_mask.fill_(False) # The source node mask in this hop.
        node_mask[subsets[-1]] = True# mark the sources
        not_visited[subsets[-1]] = False # mark visited nodes
        torch.index_select(node_mask, 0, row, out=edge_mask) # indices of all neighbors

        neighbors = col[edge_mask].unique() # remove repeats
        neighbor_mask = row.new_empty(num_nodes, dtype=torch.bool) # mask of all neighbor nodes
        edge_mask_hop = row.new_empty(row.size(0), dtype=torch.bool) # selected neighbor mask in this hop
        neighbor_mask.fill_(False)
        neighbor_mask[neighbors] = True
        neighbor_mask = torch.logical_and(neighbor_mask, not_visited) # all neighbors that are not visited
        ind = torch.where(neighbor_mask == True) # indicies of all the unvisited neighbors

        if ind[0].size(0) > max_nodes_per_hop:
            perm = torch.randperm(ind[0].size(0))
            ind = ind[0][perm]
            neighbor_mask[ind[max_nodes_per_hop:]] = False # randomly select max_nodes_per_hop nodes
            torch.index_select(neighbor_mask, 0, col, out = edge_mask_hop) # find the indicies of selected nodes
            edge_mask = torch.logical_and(edge_mask, edge_mask_hop) # change edge_mask
        subsets.append(col[edge_mask_hop])

    subset, inv = torch.cat(subsets).unique(return_inverse=True)
    inv = inv[:node_idx.numel()]

    node_mask.fill_(False)
    node_mask[subset] = True
    edge_mask = node_mask[row] & node_mask[col]

    edge_index = edge_index[:, edge_mask]

    node_idx = row.new_full((num_nodes, ), -1)
    node_idx[subset] = torch.arange(subset.size(0), device=row.device)
    edge_index = node_idx[edge_index]

    return subset, edge_index, inv, edge_mask

def minus_edge(data_observed, label, pos_edge, use_drnl):
    nodes, pos_edge_index, mapping, _ = k_hop_subgraph(node_idx=pos_edge, 
                                                       num_hops=2, 
                                                       edge_index=data_observed.edge_index,
                                                       max_nodes_per_hop=100, 
                                                       num_nodes=data_observed.num_nodes)
    x_sub = data_observed.x[nodes, :]

    # edge_mask marks the edge under pertubation, i.e., the candidate edge for LP
    edge_mask = torch.ones(pos_edge_index.size(1), dtype=torch.bool)
    ind = torch.where((pos_edge_index == mapping.view(-1, 1)).all(dim=0))
    edge_mask[ind[0]] = False
    ind = torch.where((pos_edge_index == mapping[[1, 0]].view(-1, 1)).all(dim=0))
    edge_mask[ind[0]] = False

    if use_drnl == True:
        num_nodes = torch.max(pos_edge_index)+1
        z = drnl_node_labeling(pos_edge_index[:, edge_mask], mapping[0], mapping[1], num_nodes)
        data = Data(edge_index = pos_edge_index, x = x_sub, z = z)
    else:
        data = Data(edge_index = pos_edge_index, x = x_sub, z = 0)
    data.edge_mask = edge_mask

    # label = 1 if the candidate link (p_edge) is positive and label=0 otherwis
    data.label = float(label)
    return data

def plus_edge(data_observed, label, pos_edge, use_drnl):
    nodes, edge_index_m, mapping, _ = k_hop_subgraph(node_idx=pos_edge, 
                                                     num_hops=2, 
                                                     edge_index=data_observed.edge_index,
                                                     max_nodes_per_hop=100, 
                                                     num_nodes=data_observed.num_nodes)
    x_sub = data_observed.x[nodes, :]
    pos_edge_index = edge_index_m
    pos_edge_index = torch.cat((pos_edge_index, mapping.view(-1, 1)), dim=1)
    pos_edge_index = torch.cat((pos_edge_index, mapping[[1, 0]].view(-1, 1)), dim=1)

    #edge_mask marks the edge under perturbation, i.e., the candidate edge for LP
    edge_mask = torch.ones(pos_edge_index.size(1),dtype=torch.bool)
    edge_mask[-1] = False
    edge_mask[-2] = False

    if use_drnl == True:
        num_nodes = torch.max(pos_edge_index)+1
        z = drnl_node_labeling(edge_index_m, mapping[0],mapping[1],num_nodes)
        data = Data(edge_index = pos_edge_index, x = x_sub, z = z)
    else:
        data = Data(edge_index = pos_edge_index, x = x_sub, z = 0)
    data.edge_mask = edge_mask

    #label = 1 if the candidate link (p_edge) is positive and label=0 otherwise
    data.label = float(label)

    return data

In [10]:
data_observed, feature_results = set_init_attribute_representation(train_data, val_data, test_data)

# sampling training negatives for every training epoch
neg_train_index = negative_sampling(edge_index=train_data.edge_index,
                                    num_nodes=train_data.num_nodes,
                                    num_neg_samples=train_data.pos_edge_label_index.size(1),
                                    method='sparse')

train_graphs = []
val_graphs = []
test_graphs = []

# run total edge.
for i in range(train_data.pos_edge_label_index.size(1)):
    train_graphs.append(minus_edge(data_observed, 1, train_data.pos_edge_label_index[:, i], False))
for i in range(neg_train_index.size(1)):
    train_graphs.append(plus_edge(data_observed, 0, neg_train_index[:, i], False))

for i in range(test_data.pos_edge_label_index.size(1)):
    test_graphs.append(plus_edge(data_observed, 1, test_data.pos_edge_label_index[:, i], False))
for i in range(test_data.neg_edge_label_index.size(1)):
    test_graphs.append(plus_edge(data_observed, 0, test_data.neg_edge_label_index[:, i], False))

for i in range(val_data.pos_edge_label_index.size(1)):
    val_graphs.append(plus_edge(data_observed, 1, val_data.pos_edge_label_index[:, i], False))
for i in range(val_data.neg_edge_label_index.size(1)):
    val_graphs.append(plus_edge(data_observed, 0, val_data.neg_edge_label_index[:, i], False))

print('Train_link:', str(len(train_graphs)),' Val_link:',str(len(val_graphs)),' Test_link:',str(len(test_graphs)))

train_loader = DataLoader(train_graphs,batch_size=4,shuffle=True)
val_loader = DataLoader(val_graphs,batch_size=4,shuffle=True)
test_loader = DataLoader(test_graphs,batch_size=4,shuffle=False)   

___Calculating VGAE embbeding___
Setp: 010 /500, Loss : 3.4462, Val_auc:0.7082
Setp: 020 /500, Loss : 1.9746, Val_auc:0.7082
Setp: 030 /500, Loss : 1.4562, Val_auc:0.7082
Setp: 040 /500, Loss : 1.3756, Val_auc:0.7082
Setp: 050 /500, Loss : 1.3599, Val_auc:0.7141
Setp: 060 /500, Loss : 1.3364, Val_auc:0.7141
Setp: 070 /500, Loss : 1.2979, Val_auc:0.7201
Setp: 080 /500, Loss : 1.2056, Val_auc:0.7229
Setp: 090 /500, Loss : 1.1766, Val_auc:0.7229
Setp: 100 /500, Loss : 1.1499, Val_auc:0.7231
Setp: 110 /500, Loss : 1.1323, Val_auc:0.7256
Setp: 120 /500, Loss : 1.1180, Val_auc:0.7257
Setp: 130 /500, Loss : 1.1164, Val_auc:0.7403
Setp: 140 /500, Loss : 1.0988, Val_auc:0.7542
Setp: 150 /500, Loss : 1.0699, Val_auc:0.7692
Setp: 160 /500, Loss : 1.0363, Val_auc:0.8115
Setp: 170 /500, Loss : 1.0171, Val_auc:0.8269
Setp: 180 /500, Loss : 0.9992, Val_auc:0.8288
Setp: 190 /500, Loss : 1.0040, Val_auc:0.8300
Setp: 200 /500, Loss : 1.0052, Val_auc:0.8367
Setp: 210 /500, Loss : 0.9956, Val_auc:0.8381
S

In [11]:
for i in test_loader:
    print(i)
    

DataBatch(x=[9437, 32], edge_index=[2, 28256], z=[4], edge_mask=[28256], label=[4], batch=[9437], ptr=[5])
DataBatch(x=[9436, 32], edge_index=[2, 28256], z=[4], edge_mask=[28256], label=[4], batch=[9436], ptr=[5])
DataBatch(x=[9436, 32], edge_index=[2, 28256], z=[4], edge_mask=[28256], label=[4], batch=[9436], ptr=[5])
DataBatch(x=[9436, 32], edge_index=[2, 28256], z=[4], edge_mask=[28256], label=[4], batch=[9436], ptr=[5])
DataBatch(x=[9437, 32], edge_index=[2, 28256], z=[4], edge_mask=[28256], label=[4], batch=[9437], ptr=[5])
DataBatch(x=[9436, 32], edge_index=[2, 28256], z=[4], edge_mask=[28256], label=[4], batch=[9436], ptr=[5])
DataBatch(x=[9436, 32], edge_index=[2, 28256], z=[4], edge_mask=[28256], label=[4], batch=[9436], ptr=[5])
DataBatch(x=[9437, 32], edge_index=[2, 28256], z=[4], edge_mask=[28256], label=[4], batch=[9437], ptr=[5])
DataBatch(x=[9437, 32], edge_index=[2, 28256], z=[4], edge_mask=[28256], label=[4], batch=[9437], ptr=[5])
DataBatch(x=[9440, 32], edge_index=[2

In [12]:
walk_len = 6
heads = 2
hidden_channels=32
lr=0.00005
weight_decay=0

torch.cuda.empty_cache()

num_features = next(iter(train_loader)).x.size(1)

z_max = 0
torch.cuda.empty_cache()
print("Dimention of features after concatenation:",num_features)
model = LinkPred(in_channels = num_features, hidden_channels = hidden_channels,\
    heads = heads, walk_len = walk_len, drnl = False,z_max = z_max, MSE= False).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)
criterion = torch.nn.BCEWithLogitsLoss()

Dimention of features after concatenation: 32


In [13]:
def train(loader,epoch):
    model.train()
    loss_epoch=0
    for data in tqdm(loader,desc="train"):  # Iterate in batches over the training dataset.        
        data = data.to(device)
        label= data.label
        out = model(data.x, data.edge_index, data.edge_mask, data.batch, data.z)
        torch.cuda.empty_cache()
        loss = criterion(out.view(-1), label)  
        optimizer.zero_grad()
        loss.backward()  
        optimizer.step()
        loss_epoch=loss_epoch+loss.item()
    return loss_epoch/len(loader)

def test(loader,data_type='test'):
    model.eval()
    scores = torch.tensor([])
    labels = torch.tensor([])
    loss_total=0
    with torch.no_grad():
        #for data in tqdm(loader,position=0,leave=True):  # Iterate in batches over the training/test dataset.
        for data in tqdm(loader,desc='test:'+data_type):  # Iterate in batches over the training/test dataset.
            data = data.to(device)
            out = model(data.x, data.edge_index, data.edge_mask, data.batch, data.z)
            loss = criterion(out.view(-1), data.label)
            out = out.cpu().clone().detach()
            scores = torch.cat((scores,out),dim = 0)
            labels = torch.cat((labels,data.label.view(-1,1).cpu().clone().detach()),dim = 0)
        scores = scores.cpu().clone().detach().numpy()
        labels = labels.cpu().clone().detach().numpy()
        loss_total=loss_total+loss.item()
        return roc_auc_score(labels, scores), average_precision_score(labels, scores),loss_total

In [14]:
Best_Val_fromloss=1e10
Final_Test_AUC_fromloss=0
Final_Test_AP_fromloss=0

Best_Val_fromAUC=0
Final_Test_AUC_fromAUC=0
Final_Test_AP_fromAUC=0

for epoch in range(0, 50):
    loss_epoch = train(train_loader,epoch)
    val_auc, val_ap, val_loss = test(val_loader,data_type='val')
    test_auc,test_ap,_ = test(test_loader,data_type='test')
    if val_loss < Best_Val_fromloss:
        Best_Val_fromloss = val_loss
        Final_Test_AUC_fromloss = test_auc
        Final_Test_AP_fromloss = test_ap

    if val_auc > Best_Val_fromAUC:
        Best_Val_fromAUC = val_auc
        Final_Test_AUC_fromAUC = test_auc
        Final_Test_AP_fromAUC = test_ap
    print(f'Epoch: {epoch:03d}, Loss : {loss_epoch:.4f},\
     Val Loss : {val_loss:.4f}, Val AUC: {val_auc:.4f},\
      Test AUC: {test_auc:.4f}, Picked AUC:{Final_Test_AUC_fromAUC:.4f}')

print(f'From loss: Final Test AUC: {Final_Test_AUC_fromloss:.4f}, Final Test AP: {Final_Test_AP_fromloss:.4f}')
print(f'From AUC: Final Test AUC: {Final_Test_AUC_fromAUC:.4f}, Final Test AP: {Final_Test_AP_fromAUC:.4f}')

train:   0%|          | 0/1801 [00:00<?, ?it/s]

tensor([[ 9.5319e-01,  0.0000e+00,  4.5833e-01,  4.7684e-07,  2.2426e-01,
          0.0000e+00,  9.2770e-01,  0.0000e+00,  4.3417e-01,  0.0000e+00,
          2.0689e-01,  0.0000e+00,  1.0009e+00,  9.8716e-01,  1.0477e+00,
          0.0000e+00,  5.9283e-01,  0.0000e+00,  3.7430e-01,  0.0000e+00,
          1.0206e+00,  0.0000e+00,  5.6531e-01,  0.0000e+00,  3.5240e-01,
          0.0000e+00,  6.2639e-02,  0.0000e+00,  6.0648e-02,  0.0000e+00,
          5.9055e-02,  0.0000e+00,  6.2155e-02,  0.0000e+00,  6.0207e-02,
          0.0000e+00,  5.8652e-02,  0.0000e+00,  0.0000e+00,  8.1340e-01,
          0.0000e+00,  4.8067e-01,  0.0000e+00,  3.2021e-01,  0.0000e+00,
          7.8616e-01,  0.0000e+00,  4.5608e-01,  0.0000e+00,  3.0112e-01,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-2.1423e-02,  1.0746e-01, -5.3406e-02,  1.0148e-01, -3.5812e-02,
  

test:val:   0%|          | 0/106 [00:00<?, ?it/s]

ValueError: Target size (torch.Size([4])) must be the same as input size (torch.Size([248]))

In [ ]:
for i in test_loader:
    print(i)
    print(i.label)
    print(i.z)
    break

DataBatch(x=[9192, 32], edge_index=[2, 28008], z=[4], edge_mask=[28008], label=[4], batch=[9192], ptr=[5])
tensor([1., 1., 1., 1.])
tensor([0, 0, 0, 0])


In [ ]:
test_df = pd.read_csv(data_path+'raw/test.csv')
test_feats = pd.read_csv(data_path+'raw/content.csv', delimiter='\t', header=None, index_col=0)
test_x = torch.tensor(test_feats.sort_index().values, dtype=torch.float)
test_id = test_df['id'].values
test_edge_index = torch.tensor(test_df[['from', 'to']].values.T)

In [ ]:
r, c = test_edge_index
edge_mask = r.new_empty(r.size(0), dtype=torch.bool)

In [ ]:
out = model(data_observed.x, test_edge_index, data.edge_mask, data.batch, data.z)


LinkPred()